<a href="https://colab.research.google.com/github/aj1365/MultiModelCNN/blob/main/MultiModelCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################### Connecting to Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
######################## Your file directory in Google Drive

%cd /content/drive/MyDrive/MMCNN

In [ ]:
from tensorflow import keras 
from tensorflow.keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization, MaxPool2D, MaxPooling1D,Add, ConvLSTM2D, LSTM, Conv1D
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.models import Model
#from keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tensorflow.keras import backend as Kb
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras.layers import AveragePooling2D
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
#from keras.utils import plot_model
import tensorflow
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
 
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from operator import truediv
 
from plotly.offline import init_notebook_mode
 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
#!pip install spectral
import spectral

In [ ]:
## GLOBAL VARIABLES
test_ratio = 0.5
#windowSize = 8

In [ ]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'Data/')
    if name == 'BrunswickS1':
        
        data = sio.loadmat(os.path.join(data_path, 'BrunswickS1.mat'))['BrunswickS1']
        labels = sio.loadmat(os.path.join(data_path, 'Brunswick_gt.mat'))['Brunswick_gt']

     else if name == 'BrunswickS2':
        
        data = sio.loadmat(os.path.join(data_path, 'BrunswickS2.mat'))['BrunswickS2]
        labels = sio.loadmat(os.path.join(data_path, 'Brunswick_gt.mat'))['Brunswick_gt']

     else if name == 'BrunswickDEM':
        
        data = sio.loadmat(os.path.join(data_path, 'BrunswickDEM.mat'))['BrunswickDEM]
        labels = sio.loadmat(os.path.join(data_path, 'Brunswick_gt.mat'))['Brunswick_gt']


    return data, labels

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
dataset = 'BrunswickS2'
X, y = loadData(dataset)

X=(X-np.min(X))/(np.max(X)-np.min(X))
X.shape, y.shape

In [ ]:
dataset = 'BrunswickS1'
X1, y = loadData(dataset)

X1=(X1-np.min(X1))/(np.max(X1)-np.min(X1))
X1.shape, y.shape

In [ ]:
dataset = 'BrunswickDEM'
X2, y = loadData(dataset)

X2=(X2-np.min(X2))/(np.max(X2)-np.min(X2))

X2=X2.reshape(667, 2323,1)
X2.shape, y.shape


In [ ]:
X11, y1 = createImageCubes(X, y, windowSize=4)

X11.shape, y1.shape

In [ ]:
X12, y2 = createImageCubes(X1, y, windowSize=8)

X12.shape, y2.shape

In [ ]:
X12 = X12.reshape((X12.shape[0],8,8,4,1))
X12.shape

In [ ]:
X13, y3 = createImageCubes(X2, y, windowSize=8)

X13.shape, y3.shape

In [ ]:
X1train, X1test, y1train, y1test = splitTrainTestSet(X11, y1, test_ratio)

X1train.shape, X1test.shape, y1train.shape, y1test.shape



In [ ]:
X2train, X2test, y2train, y2test = splitTrainTestSet(X12, y2, test_ratio)

X2train.shape, X2test.shape, y2train.shape, y2test.shape


In [ ]:
X3train, X3test, y3train, y3test = splitTrainTestSet(X13, y3, test_ratio)

X3train.shape, X3test.shape, y3train.shape, y3test.shape


In [ ]:
ytrain = tensorflow.keras.utils.to_categorical(y1train)
ytrain.shape


In [ ]:
output_units = 11 

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras


In [ ]:
from tensorflow.keras import layers

In [ ]:
######################## Swin Transformer settings (Try to change it)

input_shape = (8, 8, 1)
patch_size = (2, 2)  # 2-by-2 sized patches
dropout_rate = 0.03  # Dropout rate
num_heads = 8  # Attention heads
embed_dim = 64  # Embedding dimension
num_mlp = 256  # MLP layer size
qkv_bias = True  # Convert embedded patches to query, key, and values with a learnable additive value
window_size = 2  # Size of attention window
shift_size = 1  # Size of shifting window
image_dimension = 8  # Initial image size

num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]


In [ ]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x, shape=(-1, patch_num_y, window_size, patch_num_x, window_size, channels)
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape=(-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        shape=(-1, patch_num_y, patch_num_x, window_size, window_size, channels),
    )
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape=(-1, height, width, channels))
    return x


class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super(DropPath, self).__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x):
        input_shape = tf.shape(x)
        batch_size = input_shape[0]
        rank = x.shape.rank
        shape = (batch_size,) + (1,) * (rank - 1)
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape, dtype=x.dtype)
        path_mask = tf.floor(random_tensor)
        output = tf.math.divide(x, 1 - self.drop_prob) * path_mask
        return output

In [ ]:
class WindowAttention(layers.Layer):
    def __init__(
        self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs
    ):
        super(WindowAttention, self).__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        num_window_elements = (2 * self.window_size[0] - 1) * (
            2 * self.window_size[1] - 1
        )
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=tf.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        self.relative_position_index = tf.Variable(
            initial_value=tf.convert_to_tensor(relative_position_index), trainable=False
        )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm=(0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(
            self.relative_position_index, shape=(-1,)
        )
        relative_position_bias = tf.gather(
            self.relative_position_bias_table, relative_position_index_flat
        )
        relative_position_bias = tf.reshape(
            relative_position_bias, shape=(num_window_elements, num_window_elements, -1)
        )
        relative_position_bias = tf.transpose(relative_position_bias, perm=(2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32
            )
            attn = (
                tf.reshape(attn, shape=(-1, nW, self.num_heads, size, size))
                + mask_float
            )
            attn = tf.reshape(attn, shape=(-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv

In [ ]:
class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super(SwinTransformer, self).__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, shape=[-1, self.window_size * self.window_size]
            )
            attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(
                mask_windows, axis=2
            )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(initial_value=attn_mask, trainable=False)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape=(-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2]
            )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, shape=(-1, self.window_size * self.window_size, channels)
        )
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows, shape=(-1, self.window_size, self.window_size, channels)
        )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
        )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2]
            )
        else:
            x = shifted_x

        x = tf.reshape(x, shape=(-1, height * width, channels))
        x = self.drop_path(x)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(PatchExtract, self).__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[0]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super(PatchEmbedding, self).__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch, delta=1)
        return self.proj(patch) + self.pos_embed(pos)


class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super(PatchMerging, self).__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape=(-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

In [ ]:
def get_cnn_model():
 
    input_shape1 =  4, 4, 12
    input_shape2 =  8, 8, 4,1
    input_shape3 =  8, 8, 1
    
 
    input1_ = Input(shape=input_shape1)
    input2_ = Input(shape=input_shape2)
    input3_ = Input(shape=input_shape3)

    

############################                   Modified VGG-16 CNN Module (Filter numbers and kernel sizes are changed)
###########        (It shoud be noted that the kernel sizes for Conv2D in the original VGG-16 are 3 by 3, kernel_size=(3,3))
    
    conv_layer1 = Conv2D(filters=32, kernel_size=(1,1), activation='relu',padding='same')(input1_)
    conv_layer2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same')(conv_layer1)
    conv_layer3 = MaxPool2D(pool_size=(1,1))(conv_layer2)
    
    conv_layer4 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer3)
    conv_layer5 = Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same')(conv_layer4)
    conv_layer6 = MaxPool2D(pool_size=(1,1))(conv_layer5)
    
    conv_layer7 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer6)
    conv_layer8 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer7)
    conv_layer9 = Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same')(conv_layer8)
    conv_layer10 = MaxPool2D(pool_size=(1,1))(conv_layer9)
    
    conv_layer11 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer10)
    conv_layer12 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer11)
    conv_layer13 = Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same')(conv_layer12)
    conv_layer14 = MaxPool2D(pool_size=(1,1))(conv_layer13)
    
    conv_layer15 = Conv2D(filters=128, kernel_size=(1,1), activation='relu',padding='same')(conv_layer14)
    conv_layer16 = Conv2D(filters=128, kernel_size=(1,1), activation='relu',padding='same')(conv_layer15)
    conv_layer17 = Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same')(conv_layer16)
    conv_layer18 = MaxPool2D(pool_size=(1,1))(conv_layer17)
    
  
 ###########################################################
######################  3D CNN
   
   

    conv_layerb1 = Conv3D(filters=64, kernel_size=(2, 2, 2), activation='relu', padding='same', name='conv1')(input2_)
    norm_1 = BatchNormalization(name='norm_a1')(conv_layerb1)
    conv_layerb2 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv2')(norm_1)
 
    
    conv3d_shape = conv_layerb2.shape
    conv_layerb2 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layerb2)

    conv_layerb3 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv3')(conv_layerb2)

    max_b_2 = MaxPool2D((2,2), strides=(2,2), padding='same')(conv_layerb3)
   
 #################################### 
    ############################### concate first  CNN to Second  CNN
    
    concate_level_1 = concatenate([conv_layer18, max_b_2])
    conv_c2 = Conv2D(32, kernel_size=(1, 1), padding='same', name='conv_2')(concate_level_1)
    norm_c2 = BatchNormalization(name='norm_2')(conv_c2)
    relu_c2 = Activation('relu', name='relu_2')(norm_c2)

 
    pool_e_1 = AveragePooling2D(pool_size=(2, 2), strides=1, padding='same', name='avg_pool_5_1')(relu_c2)
    

    flatten_1=Flatten()(pool_e_1)
    
    dense_layer1 = Dense(units=100, activation='relu')(flatten_1)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=50, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    
    ######################################## Swin Transformer
    
    x = layers.RandomCrop(image_dimension, image_dimension)(input3_)
    x = layers.RandomFlip("horizontal")(x)
    x = PatchExtract(patch_size)(x)
    x = PatchEmbedding(num_patch_x * num_patch_y, embed_dim)(x)
    x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=0,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
     )(x)
    x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=shift_size,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
     )(x)
    x = PatchMerging((num_patch_x, num_patch_y), embed_dim=embed_dim)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(50, activation="softmax")(x)
    
    #############################################
    concate_level_3 = concatenate([x, dense_layer2])

    output_layer = Dense(units=output_units, activation='softmax')(concate_level_3)
 
    model = Model(inputs=[input1_,input2_,input3_], outputs=output_layer)
    model.summary()
    
    plot_model(model, to_file='CNN_TransformersBrunswick-VGG16.png', show_shapes=True, show_layer_names=True)
 

    return model

In [ ]:
model = get_cnn_model()

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("VGG16.h5",save_best_only=True)
history = model.fit(x=[X1train,X2train,X3train], y=ytrain, batch_size = 32, epochs=100,callbacks=model_checkpoint_callback)


In [ ]:
plt.figure(figsize=(7,7)) 
plt.grid() 
plt.plot(history.history['loss'])

plt.savefig('BrunswickLoss-VGG16.tiff',facecolor='w', dpi=500)

In [ ]:
X1test = X1test.reshape(-1, 4, 4, 12)
X2test = X2test.reshape(-1, 8, 8, 4,1)
X3test = X3test.reshape(-1, 8, 8, 1)

X1test.shape,X2test.shape,X3test.shape

In [ ]:
ytest = np_utils.to_categorical(y1test)

ytest.shape

In [ ]:
Y_pred_test = model.predict([X1test,X2test,X3test])
y_pred_test = np.argmax(Y_pred_test, axis=1)


In [ ]:
ca = np.sum(y_pred_test == np.argmax(ytest, axis=1)) / ytest.shape[0]

print("Classification accuracy: %.5f" % ca)

In [ ]:
classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)